In [1]:
%matplotlib qt

from pathlib import Path
import io
import importlib
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import palettable
sys.path.insert(0, os.path.abspath('nb'))
import src.vcnmodel.VS_data as VS_data
import matplotlib.pyplot as mpl
from matplotlib import scale as mscale
from  pylibrary.plotting import plothelpers as PH
import toml
config = toml.load(open("../wheres_my_data.toml", "r"))
from matplotlib import transforms as mtransforms
from matplotlib import ticker
import plotnine as PN
def reset_style():
#     sns.set_style(rc={"pdf.fonttype": 42})
    mpl.style.use("~/.matplotlib/figures.mplstyle")
        

In [5]:
importlib.reload(VS_data)
datas = VS_data.data
sio = io.StringIO(datas)
df = pd.read_table(sio, sep=",")

# cell_inputs = {'5': 'subthreshold', '2': "weak",  '6': "weak", "9": "one", "10": "subthreshold",
#                "17": "two", "11": "three",
#                 "13": "one", "17": "two", "18": "one", "30": "weak"}

cell_inputs = {'5': 'subthreshold', '2': 'subthreshold',  '6': 'subthreshold', "9": "one", "10": "subthreshold",
               "17": "two", "11": "two",
                "13": "one", "17": "two", "18": "one", "30": 'subthreshold'}


def label(d):
#     if d['Cell'] in [2, 6, 30]:
#           return('weak')
    if d['Cell'] in [5, 10, 2, 6, 30]:
        return('subthreshold')
    elif d['Cell'] in [9, 13, 18]:
        return('one')
#     elif d['Cell'] in [11]:
#         return 'three'
    elif d['Cell'] in [11,17]:
        return 'two'
    else:
        raise ValueError('Failed to map cell number to strength')
df['strength'] = df.apply(label, axis=1)
df = df.sort_values(["Cell", "frequency", "Configuration"], ascending = (True, True, True))
df = df.reset_index()

reset_style()
# print(dir(PN))
# plotwhat = "phasesdtime"
# plotwhat = 'phasesd'
plotwhat = 'VectorStrength'
# plotwhat = 'SpikeCount'
# plotwhat = 'logSpikeCount'
# print(len(df['frequency']))

if plotwhat == 'phasesdtime':
    df['phasesdtime'] = 1e3*df['phasesd']/(2*np.pi*df['frequency'])
    df['AN_phasesdtime'] = 1e3*df['AN_phasesd']/(2*np.pi*df['frequency'])
# #     print(len(df['phasedtime']))

if plotwhat == 'logSpikeCount':
    df['logSpikeCount'] = np.log10(df['SpikeCount'])
# for i, x in enumerate(df[plotwhat]):
# #         print(df['Cell'])
# #         print(i, x)
#     print(f"{df['Cell'][i]:3d}, {df['frequency'][i]:6.1f} {df['SpikeCount'][i]:.4f}, {df[plotwhat][i]:>10.3f}")
print(df.head(20))

    index  Cell  Configuration  carrierfreq  frequency   dmod    dB  \
0      18     2            all      16000.0      100.0  100.0  30.0   
1      19     2    largestonly      16000.0      100.0  100.0  30.0   
2      20     2  removelargest      16000.0      100.0  100.0  30.0   
3      12     2            all      16000.0      200.0  100.0  30.0   
4       0     2    largestonly      16000.0      200.0  100.0  30.0   
5       1     2  removelargest      16000.0      200.0  100.0  30.0   
6      13     2            all      16000.0      300.0  100.0  30.0   
7       2     2    largestonly      16000.0      300.0  100.0  30.0   
8       3     2  removelargest      16000.0      300.0  100.0  30.0   
9      14     2            all      16000.0      400.0  100.0  30.0   
10      4     2    largestonly      16000.0      400.0  100.0  30.0   
11      5     2  removelargest      16000.0      400.0  100.0  30.0   
12     15     2            all      16000.0      500.0  100.0  30.0   
13    

In [3]:
class SquareRootScale(mscale.ScaleBase):
    """
    ScaleBase class for generating square root scale.


    Pulled from a stack overflow answer:
    https://stackoverflow.com/questions/42277989/square-root-scale-using-matplotlib-python

    Usage:
        fig, ax = plt.subplots(1)

        ax.plot(np.arange(0, 9)**2, label='$y=x^2$')
        ax.legend()

        ax.set_yscale('squareroot')
        ax.set_yticks(np.arange(0,9,2)**2)
        ax.set_yticks(np.arange(0,8.5,0.5)**2, minor=True)

        plt.show()

    Because this registers the class in the matplotlib.scale module, it should be
    just "usable" by specifying the 'squareroot' scale.

    """

    name = "squareroot"

    def __init__(self, axis, **kwargs):
         super().__init__(axis)
#             mscale.ScaleBase.__init__(self)

    def set_default_locators_and_formatters(self, axis):
        axis.set_major_locator(ticker.AutoLocator())
        axis.set_major_formatter(ticker.ScalarFormatter())
        axis.set_minor_locator(ticker.NullLocator())
        axis.set_minor_formatter(ticker.NullFormatter())

    def limit_range_for_scale(self, vmin, vmax, minpos):
        return max(0.0, vmin), vmax

    class SquareRootTransform(mtransforms.Transform):
        input_dims = 1
        output_dims = 1
        is_separable = True

        def transform_non_affine(self, a):
            return np.array(a) ** 2

        def inverted(self):
            return SquareRootScale.InvertedSquareRootTransform()

    class InvertedSquareRootTransform(mtransforms.Transform):
        input_dims = 1
        output_dims = 1
        is_separable = True

        def transform(self, a):
            return np.array(a) ** 0.5

        def inverted(self):
            return SquareRootScale.SquareRootTransform()

    def get_transform(self):
        return self.SquareRootTransform()


mscale.register_scale(SquareRootScale)

In [8]:
"""
Plot selected cells for a poster, 1 column, two rows
"""

def getCell(cell):
    return(f"VCN_c{int(cell):02d}")
# df = df.sort_values('Cell')
labels=set(df['frequency'])
labels = [str(int(l)) for l in sorted(labels)]

def scalefun(x):
    return [f"{y:.2f}" for y in x]

# filter the data to just cells 9 and 10, and the standard configuration "all"

# df = df[df["Cell"].isin([9, 10])] 
# further on configuration
df = df[df["Configuration"].isin(["all"])]

f, axs = mpl.subplots(1, 2, figsize=(12, 6))
ax = axs[0]
ax.set_yscale('squareroot')
sns.lineplot(data=df, 
    x="frequency", y=plotwhat,
    hue="strength", 
             marker='o', markersize=9,
                hue_order=['subthreshold', 'one', 'two'],
                 sizes=(42, 216),
#                size="strength",
                 ax=ax, palette="colorblind",
            # estimator=None,
)
# ax.plot(df['frequency'], df['AN_VS'], 'r--')
sns.lineplot(data=df,
            x="frequency", y="AN_VS",
            hue_order=['subthreshold', 'one', 'two'],
                 sizes=(42, 216),
#             size="strength",
             ax=ax, palette="husl",
            # estimator=None,
)


ax.set_xlim(50, 1200)
ax.set_xscale("log")
ax.set_ylim((0, 1))

ax2 = axs[1]

# ax2.set_yscale('squareroot')
sns.lineplot(data=df, 
    x="frequency", y='phasesd',
    hue="strength", 
             marker='o', markersize=9,
                hue_order=['subthreshold', 'one', 'two'],
                sizes=(42, 216),
#                size="strength",
                 ax=ax2, palette="colorblind",
            # estimator=None,
)

print(df['phasesd'])

ax2.set_xlim(80, 1100)
ax2.set_xscale("log")
ax2.set_ylim((0, 0.4))

save_file="Compare_VS_Cells.pdf"
# mpl.savefig(
#     Path(config["baseDataDirectory"], "Figures", save_file),
#     metadata={
#         "Creator": "Paul Manis",
#         "Author": "Paul Manis",
#         "Title": "SBEM Project Figure 6 Modeling : Vector Strength Summary-cells9-10",
#     },
# )
     
mpl.show()


/var/folders/mz/x2vf3ct51tbdl8ycs54m3cph0000gn/T/ipykernel_27083/1504725540.py:58: RuntimeWarning: invalid value encountered in sqrt
/var/folders/mz/x2vf3ct51tbdl8ycs54m3cph0000gn/T/ipykernel_27083/1504725540.py:58: RuntimeWarning: invalid value encountered in sqrt


0      1.3101
3      0.5342
6      0.8546
9      0.9513
12     1.1357
        ...  
195    0.8238
198    0.9625
201    1.1491
204    1.7225
207    2.3274
Name: phasesd, Length: 70, dtype: float64


In [9]:
"""
Plot all the cells in two columns using ggplot
"""

def getCell(cell):
    return(f"VCN_c{int(cell):02d}")
# df = df.sort_values('Cell')
labels=set(df['frequency'])
labels = [str(int(l)) for l in sorted(labels)]

def scalefun(x):
    return [f"{y:.2f}" for y in x]

anpt = "AN_phasesd"
if plotwhat == 'phasesdtime':
    yscale = [0.0, 1.5]
    anpt = 'AN_phasesdtime'
elif plotwhat == 'SpikeCount':
    yscale = [0, 20000]
elif plotwhat == 'logSpikeCount':
    yscale = [0, 5]
elif plotwhat == 'phasesd':
    yscale = [0, 2*np.pi]
    anpt = "AN_phasesd"
else:
    yscale = [-0.1, 1.0]
    anpt = 'AN_VS'
    
dodge = PN.positions.position_dodge(width=0.6)
themer = PN.themes.theme
gg = (PN.ggplot(df,
            PN.aes(x='factor(frequency)',
                           y=plotwhat,
                           group='factor(frequency)',
                           color='factor(Configuration)',
                          ))
             + PN.scale_x_discrete()
#              + PN.scales.scale_y_log10()

             + PN.geom_point(
                   PN.aes(
                        x='factor(frequency)',
                        y=plotwhat,
                        inherit=True,
#                         size=1.5,
                    ),
                    stroke=0,
                    position=PN.positions.position_dodge2(width=1),
                 #position=PN.positions.position_jitterdodge(jitter_width=1, dodge_width=1),
                    alpha=1, 
                   size=2.5,
             )
              +PN.labs(x="Frequency (Hz)", y=plotwhat, 
                       color="Synapse Configuration")
               +PN.geom_line(
                  PN.aes(x='factor(frequency)',
                  y=plotwhat,
                  ),
                    position=PN.positions.position_dodge2(width=1),
                    alpha=1,
                    color='grey',
               )
            + PN.geom_line(
                  PN.aes(x='factor(frequency)',
                  y = anpt,
                    ),
                  position = PN.positions.position_dodge2(width=1),
                alpha=1,
                color = 'red',
              )
               + PN.scales.scale_colour_brewer(type="qual", palette="Dark2")
               + PN.facet_wrap('Cell', labeller=getCell, nrow=5, ncol=2, dir='v')
               + PN.theme_minimal() # (style='whitegrid')
      + PN.theme(text=PN.element_text(family='Arial', ))
      + PN.themes.theme(panel_grid_minor = PN.element_blank())
      + PN.themes.theme(panel_grid_major = PN.element_blank())
      + PN.themes.theme(panel_spacing_x = 0.5,)
      + PN.themes.theme(panel_spacing_y = 0.25,)

      + PN.themes.theme(axis_line = PN.element_rect())
      + PN.themes.theme(axis_line = PN.element_rect())
      
      + PN.themes.theme(axis_ticks_length=20,)
      + PN.themes.theme(axis_ticks_length_minor=0,)
      + PN.themes.theme(axis_ticks_length_major=3,)
      + PN.themes.theme(axis_ticks_direction='in')
      + PN.themes.theme(axis_ticks=PN.element_line(size=1),)
      + PN.themes.theme(axis_ticks_pad=20,)
  
      + PN.themes.theme(axis_ticks_major=PN.element_line(color='k'),)
      + PN.themes.theme(axis_ticks_major_x=PN.element_line(size=1), ) 
      + PN.themes.theme(axis_ticks_major_y=PN.element_line(color='k'),)
  
      + PN.themes.theme(axis_ticks_minor=PN.element_line(color='k'),)
      + PN.themes.theme(axis_ticks_minor_x=PN.element_line(size=0),   )
      + PN.themes.theme(axis_ticks_minor_y=PN.element_line(color='k'),)
      
      + PN.themes.theme(axis_text_x=PN.element_text(angle=45))
      + PN.themes.theme(figure_size=(5, 8))
#       + PN.scale_x_discrete(labels=labels)

      + PN.scales.ylim(yscale) 

#       + PN.scales.scale_y_continuous(labels=[f"{f:.2f}" for f in [0, 0.25, 0.5, 0.75, 1.0]]) # scalefun) # PN.scales.number_format(accuracy = 0.01))
      
     )
# print(dir(PN.scales))

fig, P = gg.draw(return_ggplot = True)

for i, ax in enumerate(P.axs):
    ax.tick_params(left=True, bottom=True)

save_file = f"Fig_M6_VS_SAM.pdf"

fig.text(
    0.99,
    0.99,
    save_file,  # .replace('_', '\_'),
    transform=fig.transFigure,
    horizontalalignment="right",
    verticalalignment="top",
)
mpl.show()
mpl.savefig(
    Path(config["baseDataDirectory"], "Figures", save_file),
    metadata={
        "Creator": "Paul Manis",
        "Author": "Paul Manis",
        "Title": "SBEM Project Figure 6 Modeling : Vector Strength Summary",
    },
)
        
# fig.savefig('/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/Figures/Fig_M6_VS_SAM.pdf')

/Users/pbmanis/Desktop/Python/vcnmodel/vcn_venv/lib/python3.8/site-packages/plotnine/geoms/geom_path.py:81: PlotnineWarning: geom_path: Each group consist of only one observation. Do you need to adjust the group aesthetic?
